In [1]:
from google.colab import files
uploaded = files.upload()

Saving training.tar.gz to training.tar.gz


In [2]:
!tar -xf training.tar.gz

In [3]:
from bs4 import BeautifulSoup

In [4]:
import os

In [5]:
files= os.listdir('/content/training/positive/')


In [6]:
lst=[]
for i in files:
  str= '/content/training/positive/'+i
  H_file= open(str,'r',encoding='utf-8')
  source_code= H_file.read()
  soup = BeautifulSoup(source_code,'html.parser')
  temp=[]
  for x in soup.find_all('p'):
    temp.append(x.get_text())
  lst.append(temp)

In [8]:
pos_lst=[]
for i in lst:
  pos_lst.append(' '.join(i))


In [10]:
pos_lst[0]

'Hyperlipidemia, hyperlipoproteinemia, or hyperlipidaemia (British English) involves abnormally elevated levels of any or all lipids and/or lipoproteins in the blood.[1] It is the most common form of dyslipidemia (which includes any abnormal lipid levels). Lipids (fat-soluble molecules) are transported in a protein capsule. The size of that capsule, or lipoprotein, determines its density. The lipoprotein density and type of apolipoproteins it contains determines the fate of the particle and its influence on metabolism. Hyperlipidemias are divided into primary and secondary subtypes. Primary hyperlipidemia is usually due to genetic causes (such as a mutation in a receptor protein), while secondary hyperlipidemia arises due to other underlying causes such as diabetes. Lipid and lipoprotein abnormalities are common in the general population, and are regarded as a modifiable risk factor for cardiovascular disease due to their influence on atherosclerosis. In addition, some forms may predis

In [13]:
title=[]
for i in files:
  str= '/content/training/positive/'+i
  H_file= open(str,'r',encoding='utf-8')
  source_code= H_file.read()
  soup = BeautifulSoup(source_code,'html.parser')
  temp=[]
  for x in soup.find_all('title'):
    temp.append(x.get_text())
  title.append(temp)

In [21]:
new_title=[]

for i in title:
  x=' '.join(i)
  new_title.append(x.split('-')[0])

In [22]:
len(new_title)

3695

In [20]:
len(pos_lst)

3695

In [24]:
import pandas as pd
df_pos= pd.DataFrame({'Text': pos_lst,'Title':new_title})

In [26]:
df_pos['Class']='Disease'

In [27]:
df_pos.head()

,Text,Title,Class
0,"Hyperlipidemia, hyperlipoproteinemia, or hyper...",Hyperlipidemia,Disease
1,"Acute myeloid leukemia (AML), also known as ac...",Acute myeloid leukemia,Disease
2,Hypertropia is a condition of misalignment of ...,Hypertropia,Disease
3,Pashayan syndrome also known as Pashayan–Proza...,Pashayan syndrome,Disease
4,Periodontitis /ˌpɛrioʊdɒnˈtaɪtɪs/ or pyorrhea ...,Periodontitis,Disease


In [28]:
files= os.listdir('/content/training/negative/')
lst=[]
for i in files:
  str= '/content/training/negative/'+i
  H_file= open(str,'r',encoding='utf-8')
  source_code= H_file.read()
  soup = BeautifulSoup(source_code,'html.parser')
  temp=[]
  for x in soup.find_all('p'):
    temp.append(x.get_text())
  lst.append(temp)

pos_lst=[]
for i in lst:
  pos_lst.append(' '.join(i))


title=[]
for i in files:
  str= '/content/training/negative/'+i
  H_file= open(str,'r',encoding='utf-8')
  source_code= H_file.read()
  soup = BeautifulSoup(source_code,'html.parser')
  temp=[]
  for x in soup.find_all('title'):
    temp.append(x.get_text())
  title.append(temp)


new_title=[]

for i in title:
  x=' '.join(i)
  new_title.append(x.split('-')[0])

import pandas as pd
df_neg= pd.DataFrame({'Text': pos_lst,'Title':new_title})

In [30]:
df_neg['Class']= 'No Disease'

In [32]:
df_pos.dtypes

Text     object
Title    object
Class    object
dtype: object

In [33]:
df_neg.dtypes

Text     object
Title    object
Class    object
dtype: object

In [34]:
df_final=pd.concat([df_pos,df_neg])

In [35]:
df_final.head()

,Text,Title,Class
0,"Hyperlipidemia, hyperlipoproteinemia, or hyper...",Hyperlipidemia,Disease
1,"Acute myeloid leukemia (AML), also known as ac...",Acute myeloid leukemia,Disease
2,Hypertropia is a condition of misalignment of ...,Hypertropia,Disease
3,Pashayan syndrome also known as Pashayan–Proza...,Pashayan syndrome,Disease
4,Periodontitis /ˌpɛrioʊdɒnˈtaɪtɪs/ or pyorrhea ...,Periodontitis,Disease


In [38]:
X=df_final['Text']
y= df_final['Class']

In [37]:
import numpy as np
import matplotlib.pyplot as plt
import re 
import nltk 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import svm
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import KFold,cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [39]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.33,random_state=42,stratify=y)

In [42]:
X_train.reset_index(inplace=True,drop=True)
X_test.reset_index(inplace=True,drop=True)

In [43]:
nltk.download('stopwords')
stop= set(stopwords.words('english'))
corpus=[]
for i in range(0,X_train.shape[0]):
  text=re.sub('[^a-zA-Z]',' ',X_train[i])
  text=text.lower()
  text=text.split()
  text= [ w for w in text if not w in stop]
  ps =PorterStemmer()
  stemmed_tokens= [ps.stem(words) for words in text]
  corpus.append(' '.join(stemmed_tokens))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
tfidf = TfidfVectorizer()
train_X= tfidf.fit_transform(corpus)

In [46]:
param_grid={'C':[0.1,1,10,100,1000],'gamma':[1,0.1,0.01,0.001,0.0001],'kernel':['rbf','linear']}
grid= GridSearchCV(svm.SVC(),param_grid,cv=4,scoring='roc_auc',return_train_score=True,refit= True,verbose=3)
grid.fit(train_X,y_train)

Fitting 4 folds for each of 50 candidates, totalling 200 fits
[CV 1/4] END C=0.1, gamma=1, kernel=rbf;, score=(train=0.999, test=0.997) total time=  58.5s
[CV 2/4] END C=0.1, gamma=1, kernel=rbf;, score=(train=0.999, test=0.999) total time=  57.1s
[CV 3/4] END C=0.1, gamma=1, kernel=rbf;, score=(train=0.999, test=0.997) total time=  57.9s
[CV 4/4] END C=0.1, gamma=1, kernel=rbf;, score=(train=0.999, test=0.998) total time=  58.8s
[CV 1/4] END C=0.1, gamma=1, kernel=linear;, score=(train=0.999, test=0.998) total time=  28.7s
[CV 2/4] END C=0.1, gamma=1, kernel=linear;, score=(train=0.999, test=0.999) total time=  28.4s
[CV 3/4] END C=0.1, gamma=1, kernel=linear;, score=(train=1.000, test=0.998) total time=  29.4s
[CV 4/4] END C=0.1, gamma=1, kernel=linear;, score=(train=0.999, test=0.998) total time=  29.0s
[CV 1/4] END C=0.1, gamma=0.1, kernel=rbf;, score=(train=0.998, test=0.997) total time=  59.9s
[CV 2/4] END C=0.1, gamma=0.1, kernel=rbf;, score=(train=0.998, test=0.999) total time=

GridSearchCV(cv=4, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'linear']},
             return_train_score=True, scoring='roc_auc', verbose=3)

In [47]:
def inference(data,predictor):
  corpus=[]
  for i in range(0,data.shape[0]):
    text=re.sub('[^a-zA-Z]',' ',data[i])
    text=text.lower()
    text=text.split()
    text= [ w for w in text if not w in stop]
    ps =PorterStemmer()
    stemmed_tokens= [ps.stem(words) for words in text]
    corpus.append(' '.join(stemmed_tokens))
  test_X= tfidf.transform(corpus)
  pred= predictor.predict(test_X)
  return pred

In [48]:
pred= inference(X_test,grid)

In [50]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

     Disease       0.98      0.98      0.98      1220
  No Disease       0.99      0.99      0.99      3300

    accuracy                           0.99      4520
   macro avg       0.99      0.99      0.99      4520
weighted avg       0.99      0.99      0.99      4520



In [69]:
X_test[0]

'Bathysolea is a genus of soles native to the eastern Atlantic and western Indian oceans. There are currently four recognized species in this genus:[1]'

In [70]:
pred[0]

'No Disease'